# Storm Research Assistant

Reference
https://github.com/langchain-ai/langgraph/blob/main/examples/storm/storm.ipynb


In [1]:
## Prereqs

In [2]:

# %pip install -U langchain_community langchain_openai langgraph wikipedia  scikit-learn  langchain_fireworks
# # We use one or the other search engine below
# %pip install -U tavily-python, playwright
# %pip install -U duckduckgo-search
# # ! apt-get install graphviz graphviz-dev
# %pip install pygraphviz



In [3]:

example_topic = "Impact of THE Red Cross Church IN Zimbabwe early history"

In [4]:
from storm import *

os.environ["LANGCHAIN_PROJECT"] = 'STORM_RESEARCHER'


fast_llm, long_context_llm = get_openai_llms(regular_model="gpt-3.5-turbo", long_context_model="gpt-3.5-turbo-0125")

# ollama_model = 'mistral:7b-instruct-q4_K_M'
# fast_llm, long_context_llm = get_ollama_llms(regular_model=ollama_model, long_context_model=ollama_model)


embeddings = get_gpt4all_embeddings()



bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


### Generate Initial Outline


In [5]:
outline_system_prompt = prompts.outline_system_wiki_writer
outline_human_prompt = prompts.outline_user_topic_formatinstructions
direct_gen_outline_prompt = get_chat_prompt_from_prompt_templates([outline_system_prompt, outline_human_prompt])

outline_parser = get_pydantic_parser(pydantic_object=Outline)
generate_outline_direct = get_chain_with_outputparser(direct_gen_outline_prompt, fast_llm, outline_parser)

In [6]:

initial_outline = generate_outline_direct.invoke({"topic": example_topic})
logger.info(initial_outline.as_str)

2024-03-31 09:40:06,678 [MainThread  ] [INFO ]  # Impact of The Red Cross Church in Zimbabwe Early History

## Introduction to The Red Cross Church in Zimbabwe

Provide an overview of the establishment and early history of The Red Cross Church in Zimbabwe.

## Founding and Mission

Detail the founding principles and mission of The Red Cross Church in Zimbabwe.

## Key Contributions to Zimbabwean Society

Explore the impact The Red Cross Church had on various aspects of society in Zimbabwe during its early history.

## Challenges Faced and Resolutions

Discuss the challenges encountered by The Red Cross Church in Zimbabwe and how they were addressed.

## Legacy and Continued Influence

Examine the lasting legacy and influence of The Red Cross Church in Zimbabwe, highlighting any ongoing impact.

## Notable Figures Associated with The Red Cross Church

Profile key individuals who played significant roles in shaping the early history of The Red Cross Church in Zimbabwe.


### Expand Topics


In [7]:
related_subjects_prompt = get_chat_prompt_from_prompt_templates([prompts.related_subjects_human_wiki_writer])
related_topics_parser = get_pydantic_parser(RelatedSubjects)
expand_chain = get_chain_with_outputparser(related_subjects_prompt, fast_llm, related_topics_parser)

In [8]:
related_subjects = await expand_chain.ainvoke({"topic": example_topic})
related_subjects

RelatedSubjects(topics=['History of Christianity in Zimbabwe', 'Red Cross Church in Zimbabwe', 'Impact of Christianity on Zimbabwean culture', 'Colonial history of Zimbabwe', 'Missionary activities in Zimbabwe', 'Religious organizations in Zimbabwe', 'Humanitarian work in Zimbabwe', 'Zimbabwean independence movement'])

## Generate Perspectives

From these related subjects, we can select representative Wiki editors as "subject matter experts" with distinct backgrounds and affiliations. These will help distribute the search process to encourage a more well-rounded final report.


In [9]:


gen_perspectives_prompt = get_chat_prompt_from_prompt_templates([prompts.perspective_system_generator, prompts.outline_user_topic_formatinstructions])
perspectives_parser = get_pydantic_parser(pydantic_object=Perspectives)

gen_perspectives_chain = gen_perspectives_prompt.partial(format_instructions=perspectives_parser.get_format_instructions()) | fast_llm | perspectives_parser



@as_runnable
async def survey_subjects(topic: str)-> Perspectives:
    logger.info(f"Survey Subjects for Topic: {topic}")
    related_subjects = await expand_chain.ainvoke({"topic": topic})
    retrieved_docs = await wikipedia_retriever.abatch(
        related_subjects.topics, return_exceptions=True
    )
    all_docs = []
    for docs in retrieved_docs:
        if isinstance(docs, BaseException):
            continue
        all_docs.extend(docs)
    logger.info(f"Retrieved {len(all_docs)} docs for Topic: {topic}")
    
    formatted = format_docs(all_docs)
    return await gen_perspectives_chain.ainvoke({"examples": formatted, "topic": topic})


In [10]:
perspectives = await survey_subjects.ainvoke(example_topic)
perspectives.dict()

2024-03-31 09:40:08,137 [MainThread  ] [INFO ]  Survey Subjects for Topic: Impact of THE Red Cross Church IN Zimbabwe early history
2024-03-31 09:40:11,762 [MainThread  ] [INFO ]  Retrieved 8 docs for Topic: Impact of THE Red Cross Church IN Zimbabwe early history


{'editors': [{'affiliation': 'International Committee of the Red Cross (ICRC)',
   'name': 'Anna Smith',
   'role': 'Humanitarian Aid Specialist',
   'description': 'Anna will focus on the historical role of the ICRC in Zimbabwe and its impact on early Zimbabwean history, specifically exploring the intersection of humanitarian efforts and the local population.'},
  {'affiliation': 'Heads of the Christian Denominations in Zimbabwe',
   'name': 'David Moyo',
   'role': 'Religious Historian',
   'description': 'David will provide insights into the influence of Christian denominations, including the Red Cross Church, on the social and religious landscape of Zimbabwe during its early history.'},
  {'affiliation': 'Zimbabwe National Archives',
   'name': 'Grace Mbeki',
   'role': 'Archivist',
   'description': "Grace will contribute by researching and presenting archival materials related to the Red Cross Church in Zimbabwe, shedding light on its activities and presence in the country's earl

## Expert Dialog

Each wikipedia writer is primed to role-play using the perspectives presented above. It will ask a series of questions of a second "domain expert" with access to a search engine. This generate content to generate a refined outline as well as an updated index of reference documents.

### Interview State

The conversation is cyclic, so we will construct it within its own graph. The State will contain messages, the reference docs, and the editor (with its own "persona") to make it easy to parallelize these conversations.


# Dialog Roles

The graph will have two participants: the wikipedia editor (generate_question), who asks questions based on its assigned role, and a domain expert (`gen_answer_chain), who uses a search engine to answer the questions as accurately as possible.


In [11]:
gen_qn_prompt = get_chat_prompt_from_prompt_templates([prompts.gen_question_system_generator, prompts.generate_messages_placeholder()])


def swap_roles(state: InterviewState, name: str) -> InterviewState:

    # Normalize name
    name = cleanup_name(name)

    logger.info(f'Swapping roles for {name}')

    converted = []
    for message in state["messages"]:
        if isinstance(message, AIMessage) and message.name != name:
            message = HumanMessage(**message.dict(exclude={"type"}))
        converted.append(message)
    
    state['messages'] = converted
    
    logger.info(f'Converted messages for {name} while swapping roles: {len(converted)} messages')
    return state


@as_runnable
async def generate_question(state: InterviewState) -> InterviewState:
    editor: Editor = state["editor"]

    name = cleanup_name(editor.name)


    logger.info(f'Generating question for {name}')

    gn_chain = (
        RunnableLambda(swap_roles).bind(name=name)
        | gen_qn_prompt.partial(persona=editor.persona)
        | fast_llm
        | RunnableLambda(tag_with_name).bind(name=name)
    )
    result:AIMessage = await gn_chain.ainvoke(state)
    state["messages"] = ([result])

    logger.info(f'Generated question for {name}')
    return state

In [12]:
intial_messages = [prompts.initial_question]
initial_state: InterviewState = {
    "editor": perspectives.editors[0],
    "messages": intial_messages,
    "references": None
}

question = await generate_question.ainvoke(initial_state)

question["messages"][0]

2024-03-31 09:40:22,044 [MainThread  ] [INFO ]  Generating question for AnnaSmith
2024-03-31 09:40:22,075 [asyncio_7   ] [INFO ]  Swapping roles for AnnaSmith
2024-03-31 09:40:22,078 [asyncio_7   ] [INFO ]  Converted messages for AnnaSmith while swapping roles: 1 messages
2024-03-31 09:40:23,103 [MainThread  ] [INFO ]  Generated question for AnnaSmith


AIMessage(content="I'm actually writing an article on the historical role of the ICRC in Zimbabwe and its impact on early Zimbabwean history. Can you provide me with some insights on how the ICRC's humanitarian efforts in Zimbabwe have influenced the local population and shaped the country's history during its early years?", response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 216, 'total_tokens': 274}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}, name='AnnaSmith')

### Answer questions

The `gen_answer_chain` first generates queries (query expansion) to answer the editor's question, then responds with citations.


In [13]:




gen_queries_prompt = get_chat_prompt_from_prompt_templates([prompts.gen_queries_system_generator, prompts.generate_messages_placeholder()])
queries_parser = get_pydantic_parser(Queries)

gen_queries_chain = get_chain_with_outputparser(gen_queries_prompt, fast_llm, queries_parser)
# gen_queries_prompt.partial(format_instructions=queries_parser.get_format_instructions()) | fast_llm | queries_parser

In [14]:

queries = await gen_queries_chain.ainvoke(
    {"messages": [HumanMessage(content=question["messages"][0].content)]}
)

queries

Queries(queries=['ICRC humanitarian efforts in Zimbabwe', 'Impact of ICRC in early Zimbabwean history', 'Historical role of ICRC in Zimbabwe'])

In [15]:
gen_answer_prompt = get_chat_prompt_from_prompt_templates([prompts.generate_answer_system_generator, prompts.generate_messages_placeholder()])
ac_parser = get_pydantic_parser(pydantic_object=AnswerWithCitations)

gen_answer_chain = get_chain_with_outputparser(gen_answer_prompt, fast_llm, ac_parser)\
    .with_config(run_name="GenerateAnswer")\
    # .with_structured_output(AnswerWithCitations, include_raw=True)                                  

In [16]:

async def gen_answer(
    state: InterviewState,
    config: Optional[RunnableConfig] = None,
    name: str = "SubjectMatterExpert",
    max_str_len: int = 15000,
):
    name = cleanup_name(name)

    logger.info(f'START - Generate answers for [{name}]')

    swapped_state = swap_roles(state, name)  # Convert all other AI messages
    
    # Generate search engine queries
    queries:Queries = await gen_queries_chain.ainvoke(swapped_state)

    logger.info(f"Got {len(queries.queries)} search engine queries for [{name}] -\n\t {queries.queries}")

    # Run search engine
    query_results = await search_engine.abatch(
        queries.queries, config, return_exceptions=True
    )
    successful_results = [
        res for res in query_results if not isinstance(res, Exception)
    ]

    all_query_results = {
        res["url"]: res["content"] for results in successful_results for res in results
    }
    
    logger.info(f"Got {len(successful_results)} search engine results for [{name}] - \n\t {all_query_results}")

    # We could be more precise about handling max token length if we wanted to here
    dumped_successful_results = json.dumps(all_query_results)[:max_str_len]
    
    logger.info(f"Dumped {len(dumped_successful_results)} characters for [{name}] - \n\t {dumped_successful_results}")
    
    # Append Questions from Wikipedia and Answers from the search engine
    ai_message_for_queries: AIMessage = get_ai_message(json.dumps(queries.as_dict()))
    
    tool_results_message = generate_human_message(dumped_successful_results)
    
    logger.debug(f"Got {ai_message_for_queries} for [{name}]")
    
    # tool_call = queries["raw"].additional_kwargs["tool_calls"][0]
    # tool_id = tool_call["id"]

    # tool_message = ToolMessage(tool_call_id=tool_id, content=dumped)
    

    swapped_state["messages"].extend([ai_message_for_queries, tool_results_message])
    
    # Only update the shared state with the final answer to avoid
    # polluting the dialogue history with intermediate messages
    try:
        generated: AnswerWithCitations = await gen_answer_chain.ainvoke(swapped_state)
        
        logger.info(f"Genreted final answer {generated} for [{name}] - \n\t {generated.as_str}")

    except Exception as e:
        logger.error(f"Error generating answer for [{name}] - {e}")
        generated = AnswerWithCitations(answer="", cited_urls=[])
    
    cited_urls = set(generated.cited_urls)
    
    # Save the retrieved information to a the shared state for future reference
    cited_references = {k: v for k, v in all_query_results.items() if k in cited_urls}
    
    formatted_message = AIMessage(name=name, content=generated.as_str)
    # Add message to shared state
    # state["messages"].append(formatted_message)
    state["messages"] = add_messages(state["messages"], [formatted_message])
    
    # Update references with cited references
    state["references"] = update_references(state["references"], cited_references)

    logger.info(f'END - generate answer for [{name}]')
    
    return state
    

In [17]:
intial_messages = [prompts.initial_question, generate_human_message(question["messages"][0].content)]

initial_state: InterviewState = {
    "editor": perspectives.editors[0],
    "messages": intial_messages,
    "references": {}
}

example_answer = await gen_answer(initial_state)
example_answer["messages"][-1].content

2024-03-31 09:40:23,973 [MainThread  ] [INFO ]  START - Generate answers for [SubjectMatterExpert]
2024-03-31 09:40:23,974 [MainThread  ] [INFO ]  Swapping roles for SubjectMatterExpert
2024-03-31 09:40:23,975 [MainThread  ] [INFO ]  Converted messages for SubjectMatterExpert while swapping roles: 2 messages
2024-03-31 09:40:25,254 [MainThread  ] [INFO ]  Got 3 search engine queries for [SubjectMatterExpert] -
	 ['ICRC humanitarian efforts in Zimbabwe', 'Impact of ICRC on local population in Zimbabwe', "ICRC's role in shaping early Zimbabwean history"]


Searching DuckDuckGo for [ICRC humanitarian efforts in Zimbabwe]
Got search engine results: 5 for [ICRC humanitarian efforts in Zimbabwe]
Searching DuckDuckGo for [Impact of ICRC on local population in Zimbabwe]
Got search engine results: 5 for [Impact of ICRC on local population in Zimbabwe]
Searching DuckDuckGo for [ICRC's role in shaping early Zimbabwean history]


2024-03-31 09:40:28,279 [MainThread  ] [INFO ]  Got 3 search engine results for [SubjectMatterExpert] - 
	 {'https://www.ifrc.org/taxonomy/term/402': 'More information: For more details, visit the Africa Hunger Crisis appeal page. For audio-visual material, visit the IFRC newsroom. To request an interview, contact: media@ifrc.org In Nairobi: Anne Macharia: +254 720 787 764 In Geneva: Tommaso Della Longa: +41 79 708 43 67 Mrinalini Santhanam: +41 76 381 50 06.', 'https://blogs.icrc.org/inspired/2023/07/27/simple-solutions-big-impact-zimbabwe-malawi/': 'The ICRC installed four open-access phone booths, enabling detained migrants to make free, unsupervised calls to their families between 8am and 3pm. Previously, they could only make phone calls to relatives and loved ones four times a year during ICRC visits. As a result of the programme, the number of calls made by migrants increased ten-fold ...', 'https://reliefweb.int/report/zimbabwe/ifrc-intensifies-response-escalating-cholera-outbre

Got search engine results: 5 for [ICRC's role in shaping early Zimbabwean history]


2024-03-31 09:40:35,141 [MainThread  ] [INFO ]  Genreted final answer answer="The International Committee of the Red Cross (ICRC) has had a significant impact on Zimbabwe through various humanitarian efforts, particularly in times of crisis. For instance, during a cholera outbreak in Zimbabwe, the ICRC intensified its response by launching emergency appeals and working closely with local partners like the Zimbabwe Red Cross Society. Additionally, the ICRC implemented innovative programs, such as installing open-access phone booths for detained migrants in Zimbabwe, allowing them to make free calls to their families, which greatly increased communication opportunities. Furthermore, the ICRC's focus on social protection and collaboration with development actors has helped vulnerable populations in Zimbabwe rebuild their lives with agency and ensure sustainable impact. Overall, the ICRC's long-standing commitment to promoting humanitarian norms and providing assistance to those affected b

"The International Committee of the Red Cross (ICRC) has had a significant impact on Zimbabwe through various humanitarian efforts, particularly in times of crisis. For instance, during a cholera outbreak in Zimbabwe, the ICRC intensified its response by launching emergency appeals and working closely with local partners like the Zimbabwe Red Cross Society. Additionally, the ICRC implemented innovative programs, such as installing open-access phone booths for detained migrants in Zimbabwe, allowing them to make free calls to their families, which greatly increased communication opportunities. Furthermore, the ICRC's focus on social protection and collaboration with development actors has helped vulnerable populations in Zimbabwe rebuild their lives with agency and ensure sustainable impact. Overall, the ICRC's long-standing commitment to promoting humanitarian norms and providing assistance to those affected by conflicts and crises has played a crucial role in shaping early Zimbabwean 

In [18]:
example_answer["messages"]

[HumanMessage(content='So you said you were writing an article on {example_topic}?'),
 HumanMessage(content="I'm actually writing an article on the historical role of the ICRC in Zimbabwe and its impact on early Zimbabwean history. Can you provide me with some insights on how the ICRC's humanitarian efforts in Zimbabwe have influenced the local population and shaped the country's history during its early years?"),
 AIMessage(content='{"queries": ["ICRC humanitarian efforts in Zimbabwe", "Impact of ICRC on local population in Zimbabwe", "ICRC\'s role in shaping early Zimbabwean history"]}', name='AI'),
 HumanMessage(content='{"https://www.ifrc.org/taxonomy/term/402": "More information: For more details, visit the Africa Hunger Crisis appeal page. For audio-visual material, visit the IFRC newsroom. To request an interview, contact: media@ifrc.org In Nairobi: Anne Macharia: +254 720 787 764 In Geneva: Tommaso Della Longa: +41 79 708 43 67 Mrinalini Santhanam: +41 76 381 50 06.", "https://

# Construct the Interview Graph

Now that we've defined the editor and domain expert, we can compose them in a graph.


In [19]:
def route_messages(state: InterviewState, name: str = "SubjectMatterExpert"):

    name = cleanup_name(name)

    logger.info(f'Routing messages for [{name}]')

    messages = state["messages"]
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and m.name == name]
    )

    if num_responses >= MAX_INTERVIEW_QUESTIONS:
        return END
    
    last_question = messages[-2]
    if last_question.content.endswith("Thank you so much for your help!"):
        return END
    
    logger.info(f'Continue asking question for [{name}] as this is not the last end of the conversation')
    return "ask_question"

In [20]:
builder = StateGraph(InterviewState)

builder.add_node("ask_question", generate_question)
builder.add_node("answer_question", gen_answer)
builder.add_conditional_edges("answer_question", route_messages)
builder.add_edge("ask_question", "answer_question")

builder.set_entry_point("ask_question")
interview_graph = builder.compile().with_config(run_name="Conduct Interviews")

In [21]:
from IPython.display import Image

# comment out if you have not installed pygraphviz
# Image(interview_graph.get_graph().draw_png())

In [22]:

final_step = None

initial_state = {
    "editor": perspectives.editors[0],
    "messages": [
        AIMessage(
            content=f"So you said you were writing an article on {example_topic}?",
            name="SubjectMatterExpert",
        )
    ],
}
async for step in interview_graph.astream(initial_state):
    name = next(iter(step))
    logger.info(f"Processing step: {name}")
    logger.debug("-- ", str(step[name]["messages"])[:300])
    if END in step:
        final_step = step

2024-03-31 09:40:35,201 [MainThread  ] [INFO ]  Generating question for AnnaSmith
2024-03-31 09:40:35,211 [asyncio_5   ] [INFO ]  Swapping roles for AnnaSmith
2024-03-31 09:40:35,211 [asyncio_5   ] [INFO ]  Converted messages for AnnaSmith while swapping roles: 1 messages


2024-03-31 09:40:36,470 [MainThread  ] [INFO ]  Generated question for AnnaSmith
2024-03-31 09:40:36,478 [MainThread  ] [INFO ]  Processing step: ask_question
2024-03-31 09:40:36,496 [MainThread  ] [INFO ]  START - Generate answers for [SubjectMatterExpert]
2024-03-31 09:40:36,496 [MainThread  ] [INFO ]  Swapping roles for SubjectMatterExpert
2024-03-31 09:40:36,496 [MainThread  ] [INFO ]  Converted messages for SubjectMatterExpert while swapping roles: 2 messages
2024-03-31 09:40:37,290 [MainThread  ] [INFO ]  Got 3 search engine queries for [SubjectMatterExpert] -
	 ['Historical role of the International Committee of the Red Cross (ICRC) in Zimbabwe', "Impact of ICRC's humanitarian efforts on early Zimbabwean history", "Intersection of ICRC's humanitarian efforts with local population in Zimbabwe"]


Searching DuckDuckGo for [Historical role of the International Committee of the Red Cross (ICRC) in Zimbabwe]
Got search engine results: 5 for [Historical role of the International Committee of the Red Cross (ICRC) in Zimbabwe]
Searching DuckDuckGo for [Impact of ICRC's humanitarian efforts on early Zimbabwean history]
Got search engine results: 5 for [Impact of ICRC's humanitarian efforts on early Zimbabwean history]
Searching DuckDuckGo for [Intersection of ICRC's humanitarian efforts with local population in Zimbabwe]


2024-03-31 09:40:40,337 [MainThread  ] [INFO ]  Got 3 search engine results for [SubjectMatterExpert] - 
	 {'https://www.britannica.com/topic/International-Committee-of-the-Red-Cross': 'International Committee of the Red Cross , international nongovernmental organization headquartered in Geneva, Switzerland, that seeks to aid victims of war and to ensure the observance of humanitarian law by all parties in conflict. The work of the ICRC in both World Wars was recognized by the Nobel Prize for Peace in both 1917 and 1944. It shared another Nobel Peace Prize with the League of ...', 'https://en.wikipedia.org/wiki/International_Committee_of_the_Red_Cross': 'The International Committee of the Red Cross (ICRC; French: Comité International de la Croix-Rouge) is a humanitarian organization based in Geneva, Switzerland, and is a three-time Nobel Prize Laureate. The organization has played an instrumental role in the development of rules of war and promoting humanitarian norms.. State parties (

Got search engine results: 5 for [Intersection of ICRC's humanitarian efforts with local population in Zimbabwe]


2024-03-31 09:40:46,820 [MainThread  ] [INFO ]  Genreted final answer answer="The International Committee of the Red Cross (ICRC) is a renowned humanitarian organization headquartered in Geneva, Switzerland. It has played a crucial role in aiding victims of war and promoting adherence to humanitarian law during conflicts. The ICRC's humanitarian efforts have been recognized with multiple Nobel Peace Prizes, highlighting its significant contributions to global peace and humanitarianism. Founded in 1863, the ICRC has a long and storied history of providing assistance in conflict zones worldwide, including Zimbabwe. In Zimbabwe, the ICRC has worked to protect the lives and dignity of victims of armed conflict and violence, providing crucial assistance to those in need. The organization's impartial, neutral, and independent stance has facilitated its mission to support populations affected by armed conflict, including in regions like Gaza. By closely collaborating with local communities an

Searching DuckDuckGo for [Historical role of the International Committee of the Red Cross (ICRC) in Zimbabwe]
Got search engine results: 5 for [Historical role of the International Committee of the Red Cross (ICRC) in Zimbabwe]
Searching DuckDuckGo for [Impact of ICRC's humanitarian efforts on early Zimbabwean history]
Got search engine results: 5 for [Impact of ICRC's humanitarian efforts on early Zimbabwean history]
Searching DuckDuckGo for [Intersection of ICRC's humanitarian efforts with local population in Zimbabwe]


2024-03-31 09:40:53,149 [MainThread  ] [INFO ]  Got 3 search engine results for [SubjectMatterExpert] - 
	 {'https://www.cambridge.org/core/journals/international-review-of-the-red-cross/article/abs/imperfect-relief-challenges-to-the-impartiality-and-identity-of-humanitarian-action/0520C35DD2132B3135BFC635098C6F7B': "6 While these four principles track the first four (of seven) Fundamental Principles of the International Red Cross and Red Crescent Movement (the Movement), the Movement's Code of Conduct notably also contains principles related to integrating with local capacities and involving beneficiaries in the management of aid. See International Committee of the Red Cross (ICRC) and IFRC, The Code of ...", 'https://www.britannica.com/topic/International-Committee-of-the-Red-Cross': 'The ICRC is now one component of a large network including national Red Cross and Red Crescent societies and the International Federation of Red Cross and Red Crescent Societies. (The Red Crescent was a

Got search engine results: 5 for [Intersection of ICRC's humanitarian efforts with local population in Zimbabwe]


2024-03-31 09:40:59,917 [MainThread  ] [INFO ]  Genreted final answer answer="The International Committee of the Red Cross (ICRC) has been a pivotal humanitarian organization, working globally to provide aid and ensure the observance of humanitarian law during conflicts. In Zimbabwe, the ICRC has collaborated with local bodies like the Zimbabwe Red Cross Society to combat challenges such as cholera outbreaks. Additionally, in response to natural disasters like the one in 2019 that affected Zimbabwean districts, the ICRC has played a significant role in providing relief and support to affected populations. The ICRC's involvement extends to installing phone booths in detention centers to facilitate communication for migrants, showcasing practical solutions to improve well-being. Moreover, the ICRC's crisis management and response during events like the COVID-19 pandemic have been instrumental in reaching vulnerable populations and providing essential support. The organization's commitmen

In [23]:
final_state = next(iter(final_step.values()))


In [24]:
final_state

{'messages': [AIMessage(content='So you said you were writing an article on Impact of THE Red Cross Church IN Zimbabwe early history?', name='SubjectMatterExpert'),
  AIMessage(content="I apologize for any confusion, but I am actually interested in the historical role of the International Committee of the Red Cross (ICRC) in Zimbabwe and its impact on early Zimbabwean history. Could you provide information on how the ICRC's humanitarian efforts intersected with the local population in Zimbabwe during that time?", response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 228, 'total_tokens': 290}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, name='AnnaSmith'),
  AIMessage(content='So you said you were writing an article on Impact of THE Red Cross Church IN Zimbabwe early history?', name='SubjectMatterExpert'),
  HumanMessage(content="I apologize for any confusion, but I am actually interested in the 

## Refine Outline

At this point in STORM, we've conducted a large amount of research from different perspectives. It's time to refine the original outline based on these investigations. Below, create a chain using the LLM with a long context window to update the original outline.


In [25]:
refine_outline_prompt = get_chat_prompt_from_prompt_templates([prompts.pmt_s_refine_outline, prompts.pmt_h_refine_outline])

# Using turbo preview since the context can get quite long
refine_outline_chain = get_chain_with_outputparser(refine_outline_prompt, fast_llm, outline_parser)\
    .with_config(run_name="Refine Outline")

# refine_outline_prompt.partial(format_instructions=outline_parser.get_format_instructions()) | long_context_llm | outline_parser

In [26]:
refined_outline = refine_outline_chain.invoke(
    {
        "topic": example_topic,
        "old_outline": initial_outline.as_str,
        "conversations": "\n\n".join(
            f"### {m.name}\n\n{m.content}" for m in final_state["messages"]
        ),
    }
)

In [27]:
logger.info(refined_outline.as_str)

2024-03-31 09:41:06,233 [MainThread  ] [INFO ]  # Impact of the International Committee of the Red Cross in Zimbabwe Early History

## Introduction to the International Committee of the Red Cross in Zimbabwe

Provide an overview of the historical role of the International Committee of the Red Cross (ICRC) in Zimbabwe and its impact on early Zimbabwean history.

## Founding and Mission of the ICRC in Zimbabwe

Detail the founding principles and mission of the ICRC in Zimbabwe, emphasizing its humanitarian efforts and commitment to upholding humanitarian law.

## Key Contributions of the ICRC in Zimbabwean Society

Explore how the ICRC's humanitarian efforts intersected with the local population in Zimbabwe, addressing challenges like cholera outbreaks, natural disasters, and communication barriers in detention centers.

## Challenges Faced and Resolutions by the ICRC

Discuss the challenges encountered by the ICRC in Zimbabwe and how the organization responded effectively, showcasing pr

In [76]:
## Generate Article

async def fetch_pages_from_refs(references: list[Document], limit: int|None = None) -> dict[str, list[Document]]:
    page_map = {}
    urls = [ref.metadata['source'] if 'source' in ref.metadata else None for ref in references]
    
    # populate page_map
    # for url in urls:
    #     if url is not None:
    #         if url not in page_map:
    #             page_map[url] = []
    
    # Get web pages
    docs: list[list[Document]] = await get_web_page_docs_from_url1.abatch(urls, tags_to_extract=[ "p", "h1", "h2", "h3"], limit=limit)
    
    # filter None docs
    
    for doc in docs:
        if doc is not None and len(doc) > 0:
            key = doc[0].metadata['source'] if 'source' in doc[0].metadata else None
            if key is not None:
                if key not in page_map:
                    page_map[key] = []
                page_map[key].extend(doc)
    
    return page_map


In [84]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=25
)


reference_docs = [
    Document(page_content=v, metadata={"source": k})
    for k, v in final_state["references"].items()
]

logger.info(f"Number of references: {len(reference_docs)}")

vectorstore = get_inmemory_db(reference_docs, embeddings)

# Get contents of the references
full_docs = await fetch_pages_from_refs(reference_docs[:], limit=10000)

f1 = list(chain.from_iterable(full_docs.values()))
full_split_docs = text_splitter.split_documents(f1)

vectorstore.add_documents(full_split_docs)

retriever = vectorstore.as_retriever()


2024-03-31 13:20:55,306 [MainThread  ] [INFO ]  Number of references: 12


Loading web page from url: https://www.britannica.com/topic/International-Committee-of-the-Red-Cross


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  7.14it/s]

Loading web page from url: https://en.wikipedia.org/wiki/International_Committee_of_the_Red_Cross

Fetching pages: 100%|##########| 1/1 [00:00<00:00, 13.26it/s]


...


Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.21s/it]


Loading web page from url: https://blogs.icrc.org/inspired/2023/07/27/simple-solutions-big-impact-zimbabwe-malawi/


Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.19s/it]


Loading web page from url: https://reliefweb.int/report/world/evaluation-icrcs-crisis-management-and-response-covid-19-pandemic-executive-summary-and-management-response-recommendations-november-2023


Fetching pages: 100%|##########| 1/1 [00:00<00:00,  6.00it/s]


In [85]:
d1 = retriever.invoke("What did the red cross do in Zimbabwe?")
print(d1)

[Document(page_content='(POWs).  In 2003, Nelson Mandela recalled how he was regularly visited by the ICRC during his long years of prison in apartheid South Africa.  “To me personally, and those who shared the experience of being political prisoners, the Red Cross was a beacon of humanity within the dark inhumane world of political imprisonment,” he said.        Members of the Red Cross wait outside a hangar in Kalamata, Greece, where migrants have been temporarily housed, following the sinking of a fishing boat carrying hundreds of passengers.   Byron Smith/Getty Images    Responding to natural disasters is another key element of the ICRC’s work.  During the height of the initial coronavirus rampage through Italy, Red Cross workers were among those going door to door in the city of Bergamo.  When an earthquake (https://www.cnn.com/2023/10/14/world/afghanistan-earthquake-herat-survivors-dst-intl-hnk/index.html) tore through northern Afghanistan earlier this year, the ICRC were among t

In [73]:



# d2 = await fetch_pages_from_refs(reference_docs[:3], limit=10000)

# for k, v in d2.items():
#     pprint.pprint(f"URL: {k}\n\n{v}")
#     print("\n\n")

#### Generate Sections

Now you can generate the sections using the indexed docs.


In [39]:


section_writer_prompt = get_chat_prompt_from_prompt_templates([prompts.pmt_s_section_writer, prompts.pmt_h_section_writer])


async def retrieve(inputs: dict):
    docs = await retriever.ainvoke(inputs["topic"] + ": " + inputs["section"])
    formatted = "\n".join(
        [
            f'<Document href="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
    return {"docs": formatted, **inputs}

wiki_parser = get_pydantic_parser(WikiSection)

section_writer = (
    retrieve
    | section_writer_prompt.partial(format_instructions=wiki_parser.get_format_instructions())
    | long_context_llm
    | wiki_parser
)

In [40]:
section = await section_writer.ainvoke(
    {
        "outline": refined_outline.as_str,
        "section": refined_outline.sections[1].section_title,
        "topic": example_topic,
    }
)
print(section.as_str)

## Founding of the Red Cross Church

The Red Cross Church in Zimbabwe has a rich history dating back to its founding in the early 20th century. The church was established during a period of social and political transformation in Zimbabwe, where various indigenous churches rose to prominence. The Red Cross Church, with its unique blend of Christian beliefs and local traditions, quickly gained a following among the local population. Its founding members, inspired by a vision of faith-based community development, set out to create a place of worship that would also serve as a hub for social welfare and humanitarian initiatives.

### Historical Background

The Red Cross Church emerged as a response to the changing religious landscape in Zimbabwe, where traditional beliefs intertwined with Christian teachings. The founders of the church sought to create a space that honored both their cultural heritage and their Christian faith, fostering a sense of unity and belonging among the congregatio

#### Generate final article

Now we can rewrite the draft to appropriately group all the citations and maintain a consistent voice.


In [ ]:
from langchain_core.output_parsers import StrOutputParser

writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert Wikipedia author. Write the complete wiki article on {topic} using the following section drafts:\n\n"
            "{draft}\n\nStrictly follow Wikipedia format guidelines.",
        ),
        (
            "user",
            'Write the complete Wiki article using markdown format. Organize citations using footnotes like "[1]","" avoiding duplicates in the footer. Include URLs in the footer.',
        ),
    ]
)

writer = writer_prompt | long_context_llm | StrOutputParser()

In [ ]:
for tok in writer.stream({"topic": example_topic, "draft": section.as_str}):
    print(tok, end="")

## Final Flow

Now it's time to string everything together. We will have 6 main stages in sequence:
.

1. Generate the initial outline + perspectives
2. Batch converse with each perspective to expand the content for the article
3. Refine the outline based on the conversations
4. Index the reference docs from the conversations
5. Write the individual sections of the article
6. Write the final wiki

The state tracks the outputs of each stage.


In [ ]:
class ResearchState(TypedDict):
    topic: str
    outline: Outline
    editors: List[Editor]
    interview_results: List[InterviewState]
    # The final sections output
    sections: List[WikiSection]
    article: str

In [ ]:
import asyncio


async def initialize_research(state: ResearchState):
    topic = state["topic"]
    coros = (
        generate_outline_direct.ainvoke({"topic": topic}),
        survey_subjects.ainvoke(topic),
    )
    results = await asyncio.gather(*coros)
    return {
        **state,
        "outline": results[0],
        "editors": results[1].editors,
    }


async def conduct_interviews(state: ResearchState):
    topic = state["topic"]
    initial_states = [
        {
            "editor": editor,
            "messages": [
                AIMessage(
                    content=f"So you said you were writing an article on {topic}?",
                    name="SubjectMatterExpert",
                )
            ],
        }
        for editor in state["editors"]
    ]
    # We call in to the sub-graph here to parallelize the interviews
    interview_results = await interview_graph.abatch(initial_states)

    return {
        **state,
        "interview_results": interview_results,
    }


def format_conversation(interview_state):
    messages = interview_state["messages"]
    convo = "\n".join(f"{m.name}: {m.content}" for m in messages)
    return f'Conversation with {interview_state["editor"].name}\n\n' + convo


async def refine_outline(state: ResearchState):
    convos = "\n\n".join(
        [
            format_conversation(interview_state)
            for interview_state in state["interview_results"]
        ]
    )

    updated_outline = await refine_outline_chain.ainvoke(
        {
            "topic": state["topic"],
            "old_outline": state["outline"].as_str,
            "conversations": convos,
        }
    )
    return {**state, "outline": updated_outline}


async def index_references(state: ResearchState):
    all_docs = []
    for interview_state in state["interview_results"]:
        reference_docs = [
            Document(page_content=v, metadata={"source": k})
            for k, v in interview_state["references"].items()
        ]
        all_docs.extend(reference_docs)
    await vectorstore.aadd_documents(all_docs)
    return state


async def write_sections(state: ResearchState):
    outline = state["outline"]
    sections = await section_writer.abatch(
        [
            {
                "outline": refined_outline.as_str,
                "section": section.section_title,
                "topic": state["topic"],
            }
            for section in outline.sections
        ]
    )
    return {
        **state,
        "sections": sections,
    }


async def write_article(state: ResearchState):
    topic = state["topic"]
    sections = state["sections"]
    draft = "\n\n".join([section.as_str for section in sections])
    article = await writer.ainvoke({"topic": topic, "draft": draft})
    return {
        **state,
        "article": article,
    }

#### Create the graph


In [ ]:
builder_of_storm = StateGraph(ResearchState)

nodes = [
    ("init_research", initialize_research),
    ("conduct_interviews", conduct_interviews),
    ("refine_outline", refine_outline),
    ("index_references", index_references),
    ("write_sections", write_sections),
    ("write_article", write_article),
]
for i in range(len(nodes)):
    name, node = nodes[i]
    builder_of_storm.add_node(name, node)
    if i > 0:
        builder_of_storm.add_edge(nodes[i - 1][0], name)

builder_of_storm.set_entry_point(nodes[0][0])
builder_of_storm.set_finish_point(nodes[-1][0])
storm = builder_of_storm.compile()

In [ ]:
async for step in storm.astream(
    {
        "topic": "Building better slack bots using LLMs",
    }
):
    name = next(iter(step))
    print(name)
    logger.info("-- ", str(step[name])[:300])
    if END in step:
        results = step

In [ ]:
article = results[END]["article"]

## Render the Wiki

Now we can render the final wiki page!


In [ ]:
from IPython.display import Markdown

# We will down-header the sections to create less confusion in this notebook
Markdown(article.replace("\n#", "\n##"))